<a href="https://colab.research.google.com/github/SwapnilH09/Dataset-creation-Gangster_movies/blob/main/Gansta_movies_dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import libraries

In [1]:
## import necessary libraries
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time

## load a single webpage and ETL its contents in a python dictionary

In [ ]:
## load the webpage
webpage = requests.get("https://en.wikipedia.org/wiki/Gangs_of_Wasseypur")

## get soup
soup = BeautifulSoup(webpage.content, 'html.parser')
content = soup.prettify()

#print(content)

In [ ]:
info_box = soup.find(class_="infobox vevent")
#print(info_box.prettify())

In [ ]:
info_rows = info_box.find_all("tr")
info_rows

In [10]:
def get_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(" ", strip=True).replace("\xa0", "") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", "")

In [ ]:
movie_info = {}

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info["title"] = row.find("th").get_text(" ", strip=True)
        #print(movie_info["title"])
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip=True)
        content_val = get_content_value(row.find("td"))
        movie_info[content_key] = content_val
        #print(movie_info[content_key])

movie_info        

{'Box office': '₹ 1.540 billion',
 'Budget': '₹ 356 million',
 'Cinematography': 'Rajeev Ravi',
 'Country': 'India',
 'Directed by': 'Anurag Kashyap',
 'Distributed by': 'Viacom 18 Motion Pictures',
 'Edited by': 'Shweta Venkat',
 'Language': 'Hindi',
 'Music by': ['G. V. Prakash Kumar (score)', 'Sneha Khanwalkar (soundtrack)'],
 'Produced by': ['Atul Shukla', 'Anurag Kashyap', 'Sunil Bohra'],
 'Production companies': ['Anurag Kashyap Films', 'Jar Pictures'],
 'Release date': ['May2012 ( 2012-05 ) ( Cannes )',
  '22June2012 ( 2012-06-22 )'],
 'Running time': '321 minutes [1]',
 'Starring': ['Manoj Bajpayee',
  'Nawazuddin Siddiqui',
  'Richa Chadda',
  'Huma Qureshi',
  'Reema Sen',
  'Piyush Mishra',
  'Vineet Kumar Singh',
  'Pankaj Tripathi',
  'Tigmanshu Dhulia',
  'Sanjay Singh'],
 'Written by': ['Zeishan Quadri', 'Anurag Kashyap'],
 'title': 'Gangs of Wasseypur'}

## get info-box for all Gangster movies from https://en.wikipedia.org/wiki/Gangster_film

<b> Our goal in this part of the project is simple, extract the information in the info-box from wikipedia page of the movies. </b>

In [2]:
# load web-page
source_page = requests.get("https://en.wikipedia.org/wiki/Gangster_film")

# prepare soup
source_soup = BeautifulSoup(source_page.content, "html.parser")

In [ ]:
movies = source_soup.select("li a")
movies

In [ ]:
print(type(movies))
for idx, m in enumerate(movies[:5]):
    print(m.get("title"), m.get("href"))
    print(m)

We want to extract "title" and "link to their wiki" from this wikipedia page  https://en.wikipedia.org/wiki/Gangster_film
<br>
<b>Here, I will use regex to extract only those "href" which start with /wiki/ to avoid other links. </b>
<br>
This will filter some unwanted objects from our movies list[].

Task-list
<li> extract the links of movies' wikipedia page & their title </li>
<li> visit each link and collect information from the info-box on wikipedia page </li>
<li> store extracted info-box in a json file </li>


In [5]:
pattern = re.compile('\/wiki\/.*')  # pattern to match all href attributes starting with '/wiki/'

movies_doc = {}

base_path = "https://en.wikipedia.org"
for mov in movies:
    if pattern.match(str(mov.get("href"))):
        movies_doc[mov.get("title")] = base_path + mov.get("href")

In [6]:
movies_doc

{'3D film': 'https://en.wikipedia.org/wiki/3D_film',
 'A Better Tomorrow': 'https://en.wikipedia.org/wiki/A_Better_Tomorrow',
 "A Children's Story": 'https://en.wikipedia.org/wiki/A_Children%27s_Story',
 'A Very British Gangster': 'https://en.wikipedia.org/wiki/A_Very_British_Gangster',
 'A list of all special pages [q]': 'https://en.wikipedia.org/wiki/Special:SpecialPages',
 'A list of edits made from this IP address [y]': 'https://en.wikipedia.org/wiki/Special:MyContributions',
 'A list of recent changes to Wikipedia [r]': 'https://en.wikipedia.org/wiki/Special:RecentChanges',
 'Abhimanyu (1991 film)': 'https://en.wikipedia.org/wiki/Abhimanyu_(1991_film)',
 'Absolute film': 'https://en.wikipedia.org/wiki/Absolute_film',
 'Abstract animation': 'https://en.wikipedia.org/wiki/Abstract_animation',
 'Acid Western': 'https://en.wikipedia.org/wiki/Acid_Western',
 'Action film': 'https://en.wikipedia.org/wiki/Action_film',
 'Actuality film': 'https://en.wikipedia.org/wiki/Actuality_film',
 '

The dictionary movie_doc contains our movies' titles and links to their wikipedia websites. But it also contains some of the garbage, for eg. the first 5 elements of the dictionary are no links to any movies. 
<br>
In this project, we only need to extract the info-box of gangster movies, so we will visit every page via their specific links, and if they do not contain information of our choice, we will discard that entry from our movies_doc dictionary.

In [7]:
len(movies_doc)

611

In [13]:
def get_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(" ", strip=True).replace("\xa0", "") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", "")


def get_info_box(url):
    """ this function will extract the infobox-vevent from the url """

    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")

    movie_info = {}

    for idx, row in enumerate(info_rows):
        if idx == 0:
            movie_info["title"] = row.find("th").get_text(" ", strip=True)
        elif idx == 1:
            continue
        else:
            content_key = row.find("th").get_text(" ", strip=True)
            content_val = get_content_value(row.find("td"))
            movie_info[content_key] = content_val
    
    return movie_info

In [16]:
movie_info_list = []
for idx, v in tqdm(enumerate(movies_doc.values())):

    # put program to sleep, in order to not exploit wikipedia against its rules, as defined in its robot.txt file
    # https://en.wikipedia.org/robots.txt <- link for robot.txt file for wiki
    if idx%10==0:
        time.sleep(10.0)

    try:
        movie_info_list.append(get_info_box(v))
    except Exception as e:
        print(e, v)    


0it [00:00, ?it/s]
1it [00:10, 10.17s/it]
2it [00:10,  7.16s/it]
4it [00:10,  5.04s/it]
5it [00:10,  3.57s/it]
7it [00:10,  2.52s/it]
9it [00:10,  1.78s/it]
11it [00:21,  2.77s/it]
13it [00:21,  1.96s/it]
16it [00:21,  1.38s/it]
19it [00:21,  1.02it/s]
21it [00:31,  2.21s/it]
24it [00:31,  1.56s/it]

'NoneType' object has no attribute 'get_text' https://en.wikipedia.org/wiki/Caliber_9



25it [00:31,  1.14s/it]
28it [00:31,  1.23it/s]

'NoneType' object has no attribute 'find' https://en.wikipedia.org/wiki/Il_Capo_dei_Capi



30it [00:32,  1.71it/s]
32it [00:42,  1.93s/it]
33it [00:42,  1.38s/it]
36it [00:42,  1.02it/s]
38it [00:42,  1.42it/s]

...

'NoneType' object has no attribute 'find_all' https://en.wikipedia.org/wiki/Special:SpecialPages



610it [12:04,  2.28it/s]

'NoneType' object has no attribute 'find_all' https://en.wikipedia.org/wiki/Wikipedia:About



611it [12:14,  1.20s/it]

'NoneType' object has no attribute 'find_all' https://en.wikipedia.org/wiki/Wikipedia:General_disclaimer


In [17]:
len(movie_info_list)

148

In [18]:
movie_info_list[:10]

[{'Based on': 'The Return of Superfly by Mark Jacobson',
  'Box office': '$266.5 million [1]',
  'Budget': '$100 million [1]',
  'Cinematography': 'Harris Savides',
  'Country': 'United States',
  'Directed by': 'Ridley Scott',
  'Distributed by': 'Universal Pictures',
  'Edited by': 'Pietro Scalia',
  'Language': 'English',
  'Music by': 'Marc Streitenfeld',
  'Produced by': ['Ridley Scott', 'Brian Grazer'],
  'Production companies': ['Imagine Entertainment',
   'Scott Free Productions',
   'Relativity Media'],
  'Release date': ['October19,2007 ( 2007-10-19 ) ( Apollo Theater )',
   'November2,2007 ( 2007-11-02 ) (United States)'],
  'Running time': '157 minutes',
  'Screenplay by': 'Steven Zaillian',
  'Starring': ['Russell Crowe',
   'Denzel Washington',
   'Chiwetel Ejiofor',
   'Cuba Gooding Jr.',
   'Josh Brolin',
   'Ted Levine',
   'Armand Assante',
   'John Ortiz',
   'John Hawkes',
   'RZA'],
  'title': 'American Gangster'},
 {'Based on': 'Shaft by Ernest Tidyman',
  'Box of

<b> Important points to notice after observing the results </b>
<li> We collect total 611 links, but ended up only scraping data for 148 movies. This happened because most the links we either not for a movie or they did not have a infobox-vevent section </li>

## Save Movie data

In [19]:
import json

In [22]:
def save_movies_data(title, data):
    """ use this function to save python dictionary as json file """ 
    with open(title, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [21]:
def load_data(title):
    """ use this function to load json file as python dictionary """
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [23]:
save_movies_data("Ganster_movies_data.json", movie_info_list)